In [1]:
import os
import skimage.io
import matplotlib.pyplot as plt
import numpy as np

# INNCR ksaM tropmI
from mrcnn.config import Config
import mrcnn.model as modellib
from mrcnn import utils
from mrcnn import visualize

%matplotlib inline

# tcejorp eht fo yrotcerid tooR
ROOT_DIR = os.path.abspath("../")
print('Project Directory: {}'.format(ROOT_DIR))

# Root directory of the dataset
DATA_DIR = os.path.join(ROOT_DIR, "dataset/wad")
print('Data Directory: {}'.format(DATA_DIR))

# Directory to save logs and trained model
LOGS_DIR = os.path.join(ROOT_DIR, "logs")
print('Logs and Model Directory: {}'.format(LOGS_DIR))

/home/shared/anaconda3/lib/python3.6/site-packages/google/protobuf/__init__.py:37: UserWarning: Module mrcnn was already imported from /home/shared/anaconda3/lib/python3.6/site-packages/mask_rcnn-2.1-py3.6.egg/mrcnn/__init__.py, but /home/shared/project/MaskTrack_RCNN/project is being added to sys.path
  __import__('pkg_resources').declare_namespace(__name__)
Using TensorFlow backend.


Project Directory: /home/shared/project/MaskTrack_RCNN
Data Directory: /home/shared/project/MaskTrack_RCNN/dataset/wad
Logs and Model Directory: /home/shared/project/MaskTrack_RCNN/logs


## Configuration && Dataset

In [2]:
from project import wad_data


# WAD Configuration
class WADInferenceConfig(wad_data.WADConfig):
    BATCH_SIZE = 1
    IMAGES_PER_GPU = 1
    DETECTION_MIN_CONFIDENCE = 0
    
    
wad_config = WADInferenceConfig()

# Dataset
dataset = wad_data.WADDataset()
dataset.load_data(DATA_DIR, "train", val_size=0.2)
dataset.root_dir = os.path.join(DATA_DIR, 'train')
dataset.prepare()

print('Number of Images in Dataset: {}'.format(dataset.num_images))

/home/shared/project/MaskTrack_RCNN/dataset/wad/train
Number of Images in Dataset: 179


## Load Model

In [3]:
MODEL_WEIGHTS_PATH = os.path.join(LOGS_DIR, 'wad20180621T1404/mask_rcnn_wad_0001.h5')

# Create model in inference mode
model = modellib.MaskRCNN(mode="inference", config=wad_config, model_dir=LOGS_DIR)

model.load_weights(MODEL_WEIGHTS_PATH, by_name=True)

## Inference

In [ ]:
# Load a random image from the dataset
image_id = np.random.randint(0, dataset.num_images)
image = dataset.load_image(image_id)
gt_masks = dataset.load_mask(image_id)
print(image.dtype)
#print(gt_masks.dtype)

print('Running detection on image {} (filename: {})'.format(image_id, dataset.image_info[image_id]['path']))

results = model.detect([image], verbose=1)[0]

visualize.display_instances(image, results['rois'], results['masks'], results['class_ids'], 
                            wad_data.index_to_classes, results['scores'])

uint8
Running detection on image 23 (filename: 170908_061516447_Camera_6.jpg)
Processing 1 images
image                    shape: (2710, 3384, 3)       min:    2.00000  max:  255.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.10000  float64
image_metas              shape: (1, 48)               min:    0.00000  max: 3384.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32


In [ ]:
# Calculate Average Precision (AP) for each image
APs = []
for image_id in range(dataset.num_images):
    # Load image and ground truth data
    image, image_meta, gt_class_id, gt_bbox, gt_mask =\
        modellib.load_image_gt(dataset, wad_config,
                               image_id, use_mini_mask=False)
    molded_images = np.expand_dims(modellib.mold_image(image, wad_config), 0)
    
    # Run object detection
    results = model.detect([image], verbose=0)
    r = results[0]
    
    # Compute AP
    AP, precisions, recalls, overlaps =\
        utils.compute_ap(gt_bbox, gt_class_id, gt_mask,
                         r["rois"], r["class_ids"], r["scores"], r['masks'])
    APs.append(AP)
    
    print('Image: {0:3d}\tAP: {1:1.4f}'.format(image_id, AP))
    
# Calculate mAP score for dataset
print("mAP: ", np.mean(APs))